## 필요 라이브러리 호출

In [12]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
from scipy.fftpack import fft
from sklearn.decomposition import PCA

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [13]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [14]:
## tag name 출력 파라미터 설정
table = 'rotor'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [15]:
name

['g1_sensor1_normal',
 'g1_sensor1_type1',
 'g1_sensor1_type2',
 'g1_sensor1_type3',
 'g1_sensor2_normal',
 'g1_sensor2_type1',
 'g1_sensor2_type2',
 'g1_sensor2_type3',
 'g1_sensor3_normal',
 'g1_sensor3_type1',
 'g1_sensor3_type2',
 'g1_sensor3_type3',
 'g1_sensor4_normal',
 'g1_sensor4_type1',
 'g1_sensor4_type2',
 'g1_sensor4_type3',
 'g2_sensor1_normal',
 'g2_sensor1_type1',
 'g2_sensor1_type2',
 'g2_sensor1_type3',
 'g2_sensor2_normal',
 'g2_sensor2_type1',
 'g2_sensor2_type2',
 'g2_sensor2_type3',
 'g2_sensor3_normal',
 'g2_sensor3_type1',
 'g2_sensor3_type2',
 'g2_sensor3_type3',
 'g2_sensor4_normal',
 'g2_sensor4_type1',
 'g2_sensor4_type2',
 'g2_sensor4_type3']

## TAG Name format 변환 

* 위의 과정에서 rotor dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

* g1 Tag Name 사용하여 예제 진행 

In [16]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[:16]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'g1_sensor1_normal','g1_sensor1_type1','g1_sensor1_type2','g1_sensor1_type3','g1_sensor2_normal','g1_sensor2_type1','g1_sensor2_type2','g1_sensor2_type3','g1_sensor3_normal','g1_sensor3_type1','g1_sensor3_type2','g1_sensor3_type3','g1_sensor4_normal','g1_sensor4_type1','g1_sensor4_type2','g1_sensor4_type3'
16


## Rotor Dataset 로드

* 데이터 로드시 전체 데이터 셋을 각각 Load

* label 설명
    * normal : 정상
    * type1 : Disk 2에 회전 불균형 (270도 위치에 볼트, 너트 부착)
    * type2 : Support 4에 지지 불균형
    * type3 : Type 1 + Type 2

In [17]:
# 데이터 로드 파라미터 설정

# tag table 이름 설정
table = 'rotor'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = quote('2006-01-02 15:04:05.000000')
# Train , validation , test 데이터 셋 설정
# 학습 데이터 시작 시간 설정
start_time = quote('2024-01-01 00:00:00')
# 학습 데이터  끝 시간 설정
end_time = quote('2024-01-01 00:02:19.999')

In [18]:
# 데이터 로드 함수
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time 설정
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # 빈 데이터 프레임 생성
    df_result = pd.DataFrame()
    
    # 각 tag name 별 데이터 보간
    
    for i in range(len(df.columns[1:])):
        
        # 시간 설정
        start = pd.to_datetime(unquote(start_time))
        end = pd.to_datetime(unquote(end_time))
        
        df_ = df.iloc[ : , [0] + list(range(i+1, i+2))].dropna()
        
        # 보간할 새로운 시간 구간 생성 (1초마다 1000개의 포인트)
        # 이 경우, 원본 데이터는 1ms 간격으로 측정되었으므로 1초 간격으로 1000개의 포인트를 생성
        # 0에서 140 구간을 생성 -> 140,000
        new_time_range = pd.date_range(start=start, end=end, freq='1ms')
        new_time_range_ = pd.date_range(start=start, end=end, freq='1s')

        # 선형 보간을 사용하여 데이터 보간
        # datetime을 숫자형으로 변환 (epoch time in seconds)
        time_numeric = pd.to_numeric(df_['TIME'])
        new_time_numeric = pd.to_numeric(new_time_range)

        value = df_[df_.columns[1:].values]

        # 선형 보간 객체 생성
        interpolator = interp1d(time_numeric, value.values.reshape(-1), kind='linear', fill_value='extrapolate')
        interpolated_values = interpolator(new_time_numeric)
        interpolated_values = np.clip(interpolated_values, min(value.values), max(value.values))

        # 데이터 프레임 생성
        df_remake = pd.DataFrame(interpolated_values.reshape(-1,1000))
        df_remake['time'] = new_time_range_
        df_remake['sensor'] = f'{df_.columns[1:].item()}'

        # 이동할 컬럼과 새로운 순서 지정
        cols = df_remake.columns.tolist()
        cols.insert(0, cols.pop(cols.index('time')))
        cols.insert(1, cols.pop(cols.index('sensor')))
        df_remake = df_remake[cols]

        # 빈 데이터 프레임에 추가
        df_result = pd.concat([df_result, df_remake], ignore_index=True)
        
    # 시간순 정렬
    df_result = df_result.sort_values(by='time').reset_index(drop=True)
        
    return df_result

In [19]:
# 데이터 로드

df = data_load(table, name, start_time, end_time, timeformat)
df

,time,sensor,0,1,2,3,4,5,6,7,...,990,991,992,993,994,995,996,997,998,999
0,2024-01-01 00:00:00,g1_sensor1_normal,-0.853307,-0.524641,-0.003741,-0.297684,-0.091203,-0.045372,-0.060902,0.508235,...,0.139246,0.696438,-0.508470,0.264728,-0.399669,-0.316289,-0.763595,-1.010909,-0.718536,-0.720669
1,2024-01-01 00:00:00,g1_sensor1_type2,0.555219,-0.153753,-0.197844,-0.972652,-0.913384,-1.390481,-1.414697,-1.586338,...,1.346492,1.246483,1.169959,1.310134,1.402345,1.329445,1.220657,1.077944,0.662889,0.673957
2,2024-01-01 00:00:00,g1_sensor1_type3,3.919664,3.713706,2.698885,1.338952,0.701167,-0.333580,-0.488003,-2.033326,...,6.453300,6.381317,6.292205,6.023239,5.512099,5.003446,3.997718,3.273269,1.813393,1.391992
3,2024-01-01 00:00:00,g1_sensor2_normal,0.048823,-0.029477,-0.004731,0.009673,0.096184,0.009673,0.009673,0.024096,...,1.224365,0.973323,0.398562,0.589916,0.520218,-0.442281,-0.729798,-1.159330,-1.270378,-1.268479
4,2024-01-01 00:00:00,g1_sensor2_type1,-1.054255,-1.173785,-1.142896,-1.357203,-0.848193,-0.319767,-1.745371,0.480831,...,0.252693,-1.117656,0.199578,-0.209522,-0.181900,-0.995153,-0.854758,0.305171,-1.650853,-0.284521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2024-01-01 00:02:19,g1_sensor1_type2,0.101884,0.586425,0.301300,0.480463,0.595780,0.716434,0.976936,0.963343,...,-0.073815,0.539034,0.110189,-0.139597,0.531263,0.664176,1.050553,0.868198,0.792942,1.197780
2236,2024-01-01 00:02:19,g1_sensor1_type1,1.904225,2.580567,2.227736,2.198483,2.072673,2.332377,2.170863,1.553462,...,0.112187,-0.015991,-0.595242,-0.885469,-0.792080,-1.389177,-1.368793,-2.281878,-1.887351,-2.443287
2237,2024-01-01 00:02:19,g1_sensor1_normal,0.917832,0.091463,0.723296,0.487312,1.017593,0.800933,0.449253,0.847483,...,0.261695,-0.025523,-0.376261,-0.542628,-0.685341,-1.057457,-0.513635,-1.085233,-0.677111,-1.447941
2238,2024-01-01 00:02:19,g1_sensor4_type2,-0.734751,-1.679474,-0.205754,-0.770393,0.273960,0.291936,0.355024,0.864662,...,2.318283,1.554217,1.711223,0.320252,0.483345,-0.423499,-1.127343,-0.805819,-1.208654,-1.151504


In [20]:
# label 매핑

# 레이블 매핑
label_mapping = {
    'normal': 0,
    'type1': 1,
    'type2': 2,
    'type3': 3
}

# 컬럼 이름에서 레이블 추출하기
def get_label(column_name):
    column_name = str(column_name)
    for key in label_mapping.keys():
        if key in column_name:
            return label_mapping[key]
    return None 

In [21]:
# 각 컬럼에 대해 레이블을 적용하여 새로운 시리즈 생성
labels = pd.Series(df['sensor']).map(get_label)

# 데이터 프레임에 label 추가
df['label'] = labels.values

print(df['label'].value_counts())

label
0    560
2    560
3    560
1    560
Name: count, dtype: int64


In [22]:
# train, validation, tets 데이터 분리

# 각 센서 별로 데이터를 뽑아서 설정
# trian 각 100개
# vaildation 각 20개
# test 각 20개
train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()

for i in range(len(df['sensor'].value_counts().index)):
    
    df_train = df[df['sensor'] == df['sensor'].value_counts().index[i]][:100].iloc[:,2:]
    df_valid = df[df['sensor'] == df['sensor'].value_counts().index[i]][100:120].iloc[:,2:]
    df_test = df[df['sensor'] == df['sensor'].value_counts().index[i]][120:].iloc[:,2:]
    
    train = pd.concat([train, df_train])
    valid = pd.concat([valid, df_valid])
    test = pd.concat([test, df_test])
    
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

print(train['label'].value_counts())
print(valid['label'].value_counts())
print(test['label'].value_counts())

label
0    400
2    400
3    400
1    400
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64


## 데이터 전처리

* 각 데이터별로 hanning window, FFT, MinMax Scaling 적용

## hanning window 함수 설정

In [23]:
# hanning window 함수 설정 
def set_hanning_window(sample_rate, df):
    
    # Hanning 윈도우 생성
    hanning_window = np.hanning(sample_rate)

    # 각 행에 Hanning 윈도우 적용
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

In [24]:
# 파라미터 설정
window_length = len(df.columns[2:-1])

train_ = set_hanning_window(window_length, train.iloc[:,:-1])
valid_ = set_hanning_window(window_length, valid.iloc[:,:-1])
test_ = set_hanning_window(window_length, test.iloc[:,:-1])

## FFT 함수 설정

In [25]:
# FFT 변환 함수
def change_fft(sample_rate, df):
    # 신호의 총 샘플 수
    N = sample_rate
    
    # 각 행에 대해 FFT 적용
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    for i in range(df.shape[0]):
        # 각 행의 FFT 계산
        yf = fft(df.iloc[i].values)
        
        # FFT 결과의 절댓값을 계산하고 정규화 (유의미한 부분만)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # FFT 결과를 데이터 프레임으로 변환
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

In [26]:
# 샘플링 주기 -> 초당 데이터 개수 
sampling_rate = len(df.columns[2:-1])

# FFT 변환
train_ = change_fft(sampling_rate, train_)
valid_ = change_fft(sampling_rate, valid_)
test_ = change_fft(sampling_rate, test_)

In [27]:
# 스케일러 설정
scaler = MinMaxScaler()

# 스케일러 적용
train_ = scaler.fit_transform(train_.values)
valid_ = scaler.transform(valid_.values)
test_ = scaler.transform(test_.values)

# 데이터 프레임 설정
train_scaled = pd.DataFrame(train_)
valid_scaled = pd.DataFrame(valid_)
test_scaled = pd.DataFrame(test_)

# label 추가
train_scaled['label'] = train['label'].values
valid_scaled['label'] = valid['label'].values
test_scaled['label'] = test['label'].values

print(train_scaled['label'].value_counts())
print(valid_scaled['label'].value_counts())
print(test_scaled['label'].value_counts())

label
0    400
2    400
3    400
1    400
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64
label
0    80
2    80
3    80
1    80
Name: count, dtype: int64


## 데이터 셋 및 로더 설정

In [28]:
class Rotor_Dataset(Dataset):

    def __init__(self, df):
        self.freq_data = df.iloc[:,:-1]
        self.label = df.iloc[:,-1:].squeeze()

    def __len__(self):
        return len(self.freq_data)

    def __getitem__(self, index):

        input_time_data = self.freq_data.iloc[index,:]
        input_time_data = torch.Tensor(input_time_data).expand(1, input_time_data.shape[0])
        label = self.label[index]

        return input_time_data, label

In [29]:
# 데이터 셋 설정 
train_ = Rotor_Dataset(train_scaled)
valid_ = Rotor_Dataset(valid_scaled)
test_ = Rotor_Dataset(test_scaled)

# 데이터 로더 설정
train_dataloader = DataLoader(train_, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_, batch_size=1, shuffle=True)

In [30]:
print(list(train_dataloader)[0][0].shape)

torch.Size([8, 1, 501])


## 학습 모델 설정 

* 1D ResNet 모델 사용

In [31]:
# 1D ResNet 모델 사용 
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [32]:
# 모델 설정 파라미터
# 학습률 
learning_rate = 0.01

# 모델 초기화
model = ResNet1D(ResidualBlock, [2, 2, 2, 2], num_classes=4).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## 모델 학습 설정

* 학습 중 validation 데이터 기준 F1 score 값이 제일 높은 모델을 저장 

In [33]:
train_loss = []
train_acc = []
total_step = len(train_dataloader)
epoch_in = trange(100, desc='training')
best_f1= 0

for epoch in epoch_in:
    model.train()
    running_loss = 0.0
    correct = 0
    total=0

    preds_ = []
    targets_ = []

    # 모델 학습 
    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()
        labels = train_data[1].to(device).long().squeeze()

        optimizer.zero_grad()

        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # label 예측 값 설정 
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)
        
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')
    
    # Epoch 마다 validation을 진행해서 가장 좋은 성능을 보이는 모델을 저장 
    with torch.no_grad():
        model.eval()
        
        for batch_idx, valid_data in enumerate(valid_dataloader):

            inputs_v = valid_data[0].to(device).float()
            labels_v = valid_data[1].to(device).long().squeeze() 
            
            outputs_v = model(inputs_v)
            
            # label 예측 값 설정 
            _,pred_v = torch.max(outputs_v, dim=1)
            target_v = labels_v.view_as(pred_v)
            
            preds_.append(pred_v)
            targets_.append(target_v)
            
        # 모든 배치에서 수집된 예측과 라벨을 합침
        preds_ = torch.cat(preds_).detach().cpu().numpy()
        targets_ = torch.cat(targets_).detach().cpu().numpy()
        
        f1score = f1_score(targets_, preds_,  average='macro')
        if best_f1 < f1score:
            best_f1 = f1score
            # 베스트 모델 저장 
            with open("./result/Rotor_1d_ResNet_General.txt", "a") as text_file:
                print('epoch=====',epoch, file=text_file)
                print(classification_report(targets_, preds_, digits=4), file=text_file)
            torch.save(model, f'./result/Rotor_1d_ResNet_General.pt') 
        epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")

training:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.323616251051426, train acc: 46.0000

train loss: 0.9860608429275454, train acc: 72.5625

train loss: 0.788141967896372, train acc: 83.3125

train loss: 0.6710549941565841, train acc: 88.1250

train loss: 0.5822780100503004, train acc: 91.3125

train loss: 0.5170890546264126, train acc: 92.7500

train loss: 0.4626576783857308, train acc: 95.2500

train loss: 0.41828711269379715, train acc: 96.3750

train loss: 0.38569233473555264, train acc: 95.6875

train loss: 0.356374800868638, train acc: 97.1250

train loss: 0.33595717485689836, train acc: 95.4375

train loss: 0.3155956726285755, train acc: 97.6875

train loss: 0.29604502652767295, train acc: 98.1250

train loss: 0.2780919258827531, train acc: 98.6250

train loss: 0.2646979931241561, train acc: 97.3750

train loss: 0.25410173746935927, train acc: 96.5625

train loss: 0.24196571283329618, train acc: 98.5000

train loss: 0.23141979091860979, train acc: 97.8125

train loss: 0.2203103987374418, train acc: 99.2500

train l

## 모델 테스트

In [34]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Rotor_1d_ResNet_General.pt') 

In [35]:
# 모델 테스트 
preds_test = []
target_test = []
with torch.no_grad():
    model_.eval()
    for batch_idx, test_data in enumerate(test_dataloader):
        inputs_t = test_data[0].to(device).float()
        labels_t =  test_data[1].to(device).long().squeeze() 
        
        outputs_t = model_(inputs_t)
        
        _,pred_t = torch.max(outputs_t, dim=1)
        targets_t = labels_t.view_as(pred_t).to(device)

        preds_test.append(pred_t)
        target_test.append(targets_t)
        
    # 모든 배치에서 수집된 예측과 라벨을 합침
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


## 모델 성능 확인

In [36]:
print(classification_report(target_test, preds_test))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97        80
           1       0.97      0.96      0.97        80
           2       1.00      1.00      1.00        80
           3       1.00      1.00      1.00        80

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320

